In [1]:
import nltk
import networkx as nx


nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [2]:
from nltk.corpus import wordnet as wn

In [3]:
# Костыль, чтобы появились другие языки

wn.synsets(b'\xe7\x8a\xac'.decode('utf-8'), lang='jpn')

[Synset('dog.n.01'), Synset('spy.n.01')]

### Шаг 1 (0.3 балла)
В базе WordNet возьмите синсет 'search.v.01'. Из всех языков, которые есть в базе, извлеките списки лемм, относящихся к этому синсету.

**NB!** Если вы возьмете в качестве стартового не этот синсет, а любой другой, это принесет вам 1 бонусный балл. Но будьте внимательны: в этом случае вам нужно будет самостоятельно регулировать ограничения на количество колексификаций из шага 2 так, чтобы в итоговом графе было не меньше 30 и не больше 60 узлов.

In [4]:
synset = wn.synset("study.v.01")
lemmas = {lang: synset.lemma_names(lang) for lang in wn.langs()}
lemmas

/usr/local/lib/python3.10/dist-packages/nltk/corpus/reader/wordnet.py:1564: UserWarning: No WordNet synset found for pos=a at offset=1498548.
  warnings.warn(f"No WordNet synset found for pos={pos} at offset={offset}.")
/usr/local/lib/python3.10/dist-packages/nltk/corpus/reader/wordnet.py:1564: UserWarning: No WordNet synset found for pos=a at offset=1505508.
  warnings.warn(f"No WordNet synset found for pos={pos} at offset={offset}.")
/usr/local/lib/python3.10/dist-packages/nltk/corpus/reader/wordnet.py:1564: UserWarning: No WordNet synset found for pos=a at offset=2002046.
  warnings.warn(f"No WordNet synset found for pos={pos} at offset={offset}.")
/usr/local/lib/python3.10/dist-packages/nltk/corpus/reader/wordnet.py:1564: UserWarning: No WordNet synset found for pos=a at offset=2917945.
  warnings.warn(f"No WordNet synset found for pos={pos} at offset={offset}.")


{'eng': ['analyze', 'analyse', 'study', 'examine', 'canvass', 'canvas'],
 'als': ['analizoj', 'diskutoj', 'ekzaminoj', 'studioj'],
 'arb': ['اختبر',
  'اختصر',
  'استجوب',
  'استنطق',
  'التمس',
  'امتحن',
  'بحث',
  'تأمل',
  'تدارس',
  'جرى_دراسات',
  'حاول',
  'حلل',
  'حلل_نفسيا',
  'حلّل',
  'درس',
  'راجع_دروسه',
  'طاف_لتصيد_أصوات_الناخبين',
  'عاين',
  'فتش',
  'فحص',
  'فحص_بدقة',
  'ناقش'],
 'bul': [],
 'cmn': ['研究'],
 'dan': [],
 'ell': ['αναλύω'],
 'fin': ['analysoida', 'tutkia'],
 'fra': ['analyser', 'examiner', 'étudier'],
 'heb': [],
 'hrv': ['analizirati',
  'ispitati',
  'ispitivati',
  'preispitati',
  'preispitivati',
  'pretresati',
  'pretresti',
  'proučavati',
  'proučiti',
  'razlagati',
  'razložiti',
  'raščlaniti',
  'raščlanjivati'],
 'isl': [],
 'ita': ['analizzare', 'esaminare', 'studiare'],
 'ita_iwn': [],
 'jpn': ['分析+する',
  '吟味+する',
  '攻究+する',
  '査問+する',
  '査閲+する',
  '検する',
  '検分+する',
  '検査+する',
  '検案+する',
  '検討+する',
  '点検+する',
  '研修+する',
  '研学+する',
  '

### Шаг 2 (0.7 балла)
Теперь, наоборот, для каждой леммы из каждого языка составьте список синсетов, к которым она относится. Из этих синсетов выберите такие, к которым относится больше 3 лемм из нашего изначального списка (надеемся, что это поможет нам выделить более устойчивые и надежные связи). Оставшиеся синсеты и станут узлами нашего графа.

In [5]:
from collections import Counter

synsets_counter = Counter(syn for lang, lemmas_list in lemmas.items() for lemma in lemmas_list for syn in wn.synsets(lemma, lang=lang))
filtered_synsets = [syn for syn, count in synsets_counter.items() if count > 4]

len(filtered_synsets)

50

### Шаг 3 (1 балл)
Теперь строим ребра. Ребро между двумя синсетами ставьте в том случае, если хотя бы в одном языке есть хотя бы одна лемма, которая относится к ним обоим. Пусть граф будет взвешенным: вес ребра будет отражать количество лемм, относящихся к обоим узлам пары.

**NB!** На этом шаге мы уже забываем про исходный список лемм из шага 1 (он нам нужен был только для отбора синсетов) и учитываем все леммы, относящиеся к отобранным узлам.

**Критерии**: 0.5 балла - ребра, 0.5 балла - вес ребер

In [6]:
graph = nx.Graph()

for syn in filtered_synsets:
    graph.add_node(syn.name())

for syn1 in filtered_synsets:
    for syn2 in filtered_synsets:
        if syn1 != syn2:
            weight = 0
            for lang in wn.langs():
                lemmas1 = set(syn1.lemma_names(lang))
                lemmas2 = set(syn2.lemma_names(lang))
                common_lemmas = lemmas1.intersection(lemmas2)
                weight += len(common_lemmas)
            if weight > 0:
                graph.add_edge(syn1.name(), syn2.name(), weight=weight)

len(graph), len(graph.edges)

(50, 817)

### Шаг 4 (3 балла)
Проанализируйте получившийся граф. Сколько получилось связных компонент? Какая у этого графа плотность? Как распределились (взвешенные) степени узлов? Какие узлы оказались центральными (попробуйте несколько метрик, например, degree centrality и eigencentrality, прокомментируйте результат)? Разбейте граф на сообщества (поиграйте с несколькими алгоритмами) и прокомментируйте результаты.

**Критерии**: 0.5 - связные компоненты, 0.5 - плотность графа, 1 - степени и центральность узлов, 1 - сообщества

In [7]:
# Связные компоненты и плотность графа

num_connected_components = nx.number_connected_components(graph)
density = nx.density(graph)

num_connected_components, density

(1, 0.6669387755102041)

Выходит, что все синсеты в графе так или иначе связаны между собой, изолированных, т.е. таких, которые бы обозначали какой-то совсем "левый" семантический концепт, никак не связанный с изучением, нет. Но оно и не удивительно: мы ведь ставили рёбра всегда, когда одно слово встречалось рядом с другим, поэтому они все и связались!

Наш граф достаточно плотный, значит, большинство синсетов связано напрямую или "близко". Можно предположить, что будет очень много близких синонимов, но это мы проверим дальше.

In [8]:
# Распределение взвешенных степеней
weighted_degrees = dict(graph.degree(weight="weight"))

weighted_degrees

{'analyze.v.01': 604,
 'analyze.v.02': 154,
 'analyze.v.03': 95,
 'analyze.v.04': 85,
 'study.v.02': 154,
 'study.v.03': 313,
 'learn.v.04': 335,
 'study.v.05': 156,
 'study.v.06': 127,
 'examine.v.02': 543,
 'probe.v.01': 340,
 'examine.v.04': 179,
 'test.v.01': 370,
 'poll.v.01': 20,
 'size_up.v.01': 258,
 'hash_out.v.01': 38,
 'try_on.v.01': 153,
 'sample.v.01': 164,
 'try.v.01': 192,
 'investigate.v.02': 264,
 'investigate.v.01': 244,
 'learn.v.02': 291,
 'interpret.v.01': 171,
 'see.v.05': 297,
 'search.v.01': 186,
 'search.v.04': 156,
 'view.v.02': 205,
 'learn.v.01': 137,
 'memorize.v.01': 106,
 'teach.v.01': 111,
 'inspect.v.01': 356,
 'check.v.01': 318,
 'determine.v.08': 351,
 'control.v.02': 213,
 'survey.v.01': 86,
 'research.v.02': 190,
 'audit.v.01': 194,
 'screen.v.02': 83,
 'quiz.v.01': 106,
 'research.v.01': 147,
 'survey.v.02': 113,
 'screen.v.03': 73,
 'screen.v.01': 139,
 'interrogate.v.02': 149,
 'review.v.03': 127,
 'visit.v.04': 118,
 'check.v.22': 201,
 'check.v

In [9]:
# Degree Centrality
degree_centrality = nx.degree_centrality(graph)

degree_centrality

{'analyze.v.01': 0.9999999999999999,
 'analyze.v.02': 0.7551020408163265,
 'analyze.v.03': 0.4693877551020408,
 'analyze.v.04': 0.3469387755102041,
 'study.v.02': 0.5102040816326531,
 'study.v.03': 0.8571428571428571,
 'learn.v.04': 0.9387755102040816,
 'study.v.05': 0.6530612244897959,
 'study.v.06': 0.5306122448979591,
 'examine.v.02': 0.9183673469387754,
 'probe.v.01': 0.9183673469387754,
 'examine.v.04': 0.7142857142857142,
 'test.v.01': 0.836734693877551,
 'poll.v.01': 0.2857142857142857,
 'size_up.v.01': 0.8775510204081632,
 'hash_out.v.01': 0.42857142857142855,
 'try_on.v.01': 0.4693877551020408,
 'sample.v.01': 0.44897959183673464,
 'try.v.01': 0.5918367346938775,
 'investigate.v.02': 0.8571428571428571,
 'investigate.v.01': 0.9183673469387754,
 'learn.v.02': 0.6938775510204082,
 'interpret.v.01': 0.5918367346938775,
 'see.v.05': 0.673469387755102,
 'search.v.01': 0.8163265306122448,
 'search.v.04': 0.7551020408163265,
 'view.v.02': 0.7142857142857142,
 'learn.v.01': 0.40816326

In [10]:
# Eigenvector Centrality
eigenvector_centrality = nx.eigenvector_centrality(graph)

eigenvector_centrality

{'analyze.v.01': 0.1924605717518631,
 'analyze.v.02': 0.14976102414125383,
 'analyze.v.03': 0.08901920527281722,
 'analyze.v.04': 0.06351982292388987,
 'study.v.02': 0.10116715850763491,
 'study.v.03': 0.16861026089102146,
 'learn.v.04': 0.18366329339233248,
 'study.v.05': 0.12846099580289072,
 'study.v.06': 0.10315034671784104,
 'examine.v.02': 0.18356797710959608,
 'probe.v.01': 0.18356797710959608,
 'examine.v.04': 0.14877190080069738,
 'test.v.01': 0.17394768518922551,
 'poll.v.01': 0.06169125936630012,
 'size_up.v.01': 0.1774612318206004,
 'hash_out.v.01': 0.094556532036979,
 'try_on.v.01': 0.10217037981092755,
 'sample.v.01': 0.09837445408400607,
 'try.v.01': 0.1281907732649217,
 'investigate.v.02': 0.17496702309554982,
 'investigate.v.01': 0.18389509059707848,
 'learn.v.02': 0.13680167835288712,
 'interpret.v.01': 0.11937802406662594,
 'see.v.05': 0.13698878289858912,
 'search.v.01': 0.16656382561378244,
 'search.v.04': 0.15436778643000154,
 'view.v.02': 0.14717145870048043,
 'l

Великолепная картина! Тут видно, что мы нашли очень много близких синонимов концепту `study` в значении "изучать что-либо (внимательно), исследовать". Ирония в том, что в итоговом графе не нашлось места самому `study.v.01` - его полностью заменил `analyze.v.01`, который и стал центральным - по всей видимости, являясь почти полным синонимом *study* в этом значении и употребляемый чаще.

In [11]:
from networkx.algorithms import community

partition_louvain = community.louvain_communities(graph)

partition_louvain

[{'analyze.v.01',
  'analyze.v.02',
  'analyze.v.03',
  'analyze.v.04',
  'learn.v.01',
  'learn.v.04',
  'memorize.v.01',
  'poll.v.01',
  'study.v.02',
  'study.v.05',
  'study.v.06',
  'teach.v.01'},
 {'determine.v.08',
  'examine.v.02',
  'interpret.v.01',
  'learn.v.02',
  'see.v.05',
  'study.v.03',
  'view.v.02'},
 {'examine.v.04',
  'hash_out.v.01',
  'interrogate.v.02',
  'investigate.v.01',
  'investigate.v.02',
  'probe.v.01',
  'research.v.01',
  'research.v.02',
  'screen.v.02',
  'screen.v.03',
  'search.v.01',
  'search.v.04'},
 {'audit.v.01',
  'check.v.01',
  'check.v.02',
  'check.v.08',
  'check.v.22',
  'control.v.02',
  'control.v.05',
  'inspect.v.01',
  'review.v.03',
  'size_up.v.01',
  'survey.v.01',
  'survey.v.02',
  'visit.v.04'},
 {'quiz.v.01',
  'sample.v.01',
  'screen.v.01',
  'test.v.01',
  'try.v.01',
  'try_on.v.01'}]

In [12]:
partition_naive = community.naive_greedy_modularity_communities(graph)

partition_naive

[frozenset({'audit.v.01',
            'check.v.01',
            'check.v.02',
            'check.v.08',
            'check.v.22',
            'control.v.02',
            'control.v.05',
            'examine.v.02',
            'hash_out.v.01',
            'inspect.v.01',
            'interrogate.v.02',
            'investigate.v.01',
            'investigate.v.02',
            'probe.v.01',
            'quiz.v.01',
            'research.v.02',
            'sample.v.01',
            'screen.v.01',
            'screen.v.02',
            'search.v.01',
            'search.v.04',
            'size_up.v.01',
            'survey.v.02',
            'test.v.01',
            'try.v.01',
            'try_on.v.01',
            'visit.v.04'}),
 frozenset({'analyze.v.01',
            'analyze.v.02',
            'analyze.v.03',
            'analyze.v.04',
            'determine.v.08',
            'examine.v.04',
            'interpret.v.01',
            'learn.v.01',
            'learn.v.02',
       

Мы попробовали два алгоритма и, в принципе, второй достаточно плох, тогда как первый неплохо выделяет кластеры синсетов. У нас есть один большой, который, кажется, не очень хорошо объединяется и туда ссыпалось всё подряд (включая центральные синсеты, кстати). Чётко выделяются кластеры, связанный с глаголами расследования, глаголами оценки знаний.

### Шаг 5 (2 балла)
Постройте точно такой же граф, только теперь ставьте ребра только в том случае, если пару синсетов объединяет не менее 5 лемм (убираем все ребра с небольшим весом в поисках наиболее устойчивых связей). Проанализируйте этот граф по той же схеме (см. шаг 4). Что изменилось? Какой из графов кажется вам более содержательным и почему?

**Критерии**: 0.5 - обновленный граф, 0.5 - подсчет всех метрик заново, 1 - комментарий

In [13]:
graph2 = nx.Graph()

for syn in filtered_synsets:
    graph2.add_node(syn.name())

for syn1 in filtered_synsets:
    for syn2 in filtered_synsets:
        if syn1 != syn2:
            weight = 0
            for lang in wn.langs():
                lemmas1 = set(syn1.lemma_names(lang))
                lemmas2 = set(syn2.lemma_names(lang))
                common_lemmas = lemmas1.intersection(lemmas2)
                weight += len(common_lemmas)
            if weight > 5:
                graph2.add_edge(syn1.name(), syn2.name(), weight=weight)

len(graph2), len(graph2.edges)

(50, 248)

In [14]:
# Связные компоненты и плотность графа

num_connected_components = nx.number_connected_components(graph2)
density = nx.density(graph2)

num_connected_components, density

(2, 0.20244897959183675)

In [15]:
connected_components = list(nx.connected_components(graph2))

connected_components

[{'analyze.v.01',
  'analyze.v.02',
  'analyze.v.03',
  'analyze.v.04',
  'audit.v.01',
  'check.v.01',
  'check.v.02',
  'check.v.08',
  'check.v.22',
  'control.v.02',
  'control.v.05',
  'determine.v.08',
  'examine.v.02',
  'examine.v.04',
  'hash_out.v.01',
  'inspect.v.01',
  'interpret.v.01',
  'interrogate.v.02',
  'investigate.v.01',
  'investigate.v.02',
  'learn.v.01',
  'learn.v.02',
  'learn.v.04',
  'memorize.v.01',
  'probe.v.01',
  'quiz.v.01',
  'research.v.01',
  'research.v.02',
  'review.v.03',
  'sample.v.01',
  'screen.v.01',
  'screen.v.02',
  'screen.v.03',
  'search.v.01',
  'search.v.04',
  'see.v.05',
  'size_up.v.01',
  'study.v.02',
  'study.v.03',
  'study.v.05',
  'study.v.06',
  'survey.v.01',
  'survey.v.02',
  'teach.v.01',
  'test.v.01',
  'try.v.01',
  'try_on.v.01',
  'view.v.02',
  'visit.v.04'},
 {'poll.v.01'}]

И тут у нас появляется один изолированный кластер из одного слова: `poll`, в отличие от первого графа -- потому что рёбер меньше. Но в целом остальные слова остались связанными между собой.

Плотность графа очень низкая, что тоже говорит о маленьком количестве прямых связей между синсетами. Получается, что слова в основном остались связанными, но теперь уже намного слабее - путь, которым можно прийти от одного синсета к другому, стал длиннее.

In [16]:
# Распределение взвешенных степеней
weighted_degrees = dict(graph2.degree(weight="weight"))

weighted_degrees

{'analyze.v.01': 550,
 'analyze.v.02': 76,
 'analyze.v.03': 60,
 'analyze.v.04': 56,
 'study.v.02': 119,
 'study.v.03': 242,
 'learn.v.04': 239,
 'study.v.05': 109,
 'study.v.06': 74,
 'examine.v.02': 472,
 'probe.v.01': 255,
 'examine.v.04': 92,
 'test.v.01': 312,
 'poll.v.01': 0,
 'size_up.v.01': 191,
 'hash_out.v.01': 6,
 'try_on.v.01': 115,
 'sample.v.01': 130,
 'try.v.01': 137,
 'investigate.v.02': 195,
 'investigate.v.01': 162,
 'learn.v.02': 231,
 'interpret.v.01': 121,
 'see.v.05': 235,
 'search.v.01': 116,
 'search.v.04': 91,
 'view.v.02': 159,
 'learn.v.01': 112,
 'memorize.v.01': 75,
 'teach.v.01': 81,
 'inspect.v.01': 314,
 'check.v.01': 258,
 'determine.v.08': 291,
 'control.v.02': 147,
 'survey.v.01': 16,
 'research.v.02': 138,
 'audit.v.01': 139,
 'screen.v.02': 33,
 'quiz.v.01': 47,
 'research.v.01': 89,
 'survey.v.02': 24,
 'screen.v.03': 17,
 'screen.v.01': 49,
 'interrogate.v.02': 68,
 'review.v.03': 53,
 'visit.v.04': 45,
 'check.v.22': 155,
 'check.v.02': 160,
 'co

In [17]:
# Degree Centrality
degree_centrality = nx.degree_centrality(graph2)

degree_centrality

{'analyze.v.01': 0.7755102040816326,
 'analyze.v.02': 0.061224489795918366,
 'analyze.v.03': 0.061224489795918366,
 'analyze.v.04': 0.061224489795918366,
 'study.v.02': 0.16326530612244897,
 'study.v.03': 0.3061224489795918,
 'learn.v.04': 0.3061224489795918,
 'study.v.05': 0.18367346938775508,
 'study.v.06': 0.12244897959183673,
 'examine.v.02': 0.5102040816326531,
 'probe.v.01': 0.3877551020408163,
 'examine.v.04': 0.14285714285714285,
 'test.v.01': 0.3877551020408163,
 'poll.v.01': 0.0,
 'size_up.v.01': 0.36734693877551017,
 'hash_out.v.01': 0.02040816326530612,
 'try_on.v.01': 0.08163265306122448,
 'sample.v.01': 0.1020408163265306,
 'try.v.01': 0.1020408163265306,
 'investigate.v.02': 0.3061224489795918,
 'investigate.v.01': 0.3061224489795918,
 'learn.v.02': 0.22448979591836732,
 'interpret.v.01': 0.1020408163265306,
 'see.v.05': 0.16326530612244897,
 'search.v.01': 0.18367346938775508,
 'search.v.04': 0.14285714285714285,
 'view.v.02': 0.18367346938775508,
 'learn.v.01': 0.16326

In [18]:
# Eigenvector Centrality
eigenvector_centrality = nx.eigenvector_centrality(graph2)

eigenvector_centrality

{'analyze.v.01': 0.3150495955823745,
 'analyze.v.02': 0.02472538935710457,
 'analyze.v.03': 0.02472538935710457,
 'analyze.v.04': 0.02472538935710457,
 'study.v.02': 0.0635799819484014,
 'study.v.03': 0.15073681615703138,
 'learn.v.04': 0.14388273995912984,
 'study.v.05': 0.06793267148920666,
 'study.v.06': 0.05741747870924782,
 'examine.v.02': 0.2867832640797852,
 'probe.v.01': 0.24787968711823802,
 'examine.v.04': 0.11404273008979061,
 'test.v.01': 0.21998329143242934,
 'poll.v.01': 7.6973651896277e-22,
 'size_up.v.01': 0.22465389927488427,
 'hash_out.v.01': 0.021370951034419404,
 'try_on.v.01': 0.02440252455128838,
 'sample.v.01': 0.044415893039571025,
 'try.v.01': 0.049101649964621956,
 'investigate.v.02': 0.2016389996258576,
 'investigate.v.01': 0.19832448083213214,
 'learn.v.02': 0.10811566556131086,
 'interpret.v.01': 0.05774971201921994,
 'see.v.05': 0.09073644042616533,
 'search.v.01': 0.1200053652801119,
 'search.v.04': 0.10261522759866408,
 'view.v.02': 0.1052571144074555,
 

Если верить распределению весов, в принципе ничего не изменилось - центральные синсеты остались такими, какими и были, разве что теперь их "центральность" несколько ниже, то есть они слабее объединяют остальные концепты.

In [19]:
partition_louvain = community.louvain_communities(graph2)

partition_louvain

[{'poll.v.01'},
 {'quiz.v.01',
  'sample.v.01',
  'screen.v.01',
  'test.v.01',
  'try.v.01',
  'try_on.v.01'},
 {'analyze.v.01',
  'analyze.v.02',
  'analyze.v.03',
  'analyze.v.04',
  'examine.v.04',
  'hash_out.v.01',
  'interrogate.v.02',
  'investigate.v.01',
  'investigate.v.02',
  'probe.v.01',
  'research.v.01',
  'research.v.02',
  'search.v.01',
  'search.v.04',
  'size_up.v.01',
  'survey.v.01'},
 {'determine.v.08',
  'examine.v.02',
  'interpret.v.01',
  'learn.v.01',
  'learn.v.02',
  'learn.v.04',
  'memorize.v.01',
  'see.v.05',
  'study.v.02',
  'study.v.03',
  'study.v.05',
  'study.v.06',
  'teach.v.01',
  'view.v.02'},
 {'audit.v.01',
  'check.v.01',
  'check.v.02',
  'check.v.08',
  'check.v.22',
  'control.v.02',
  'control.v.05',
  'inspect.v.01',
  'review.v.03',
  'screen.v.02',
  'screen.v.03',
  'survey.v.02',
  'visit.v.04'}]

Прям очень красивые кластеры, в которых слова очень чётко объединились по их близости! Теперь у нас есть больше релевантных кластеров, связанных непосредственно с учёбой, исследованиями, тестированием и оценкой знаний.

Итак, выходит, что второй граф получился значительно содержательнее первого: в нём меньше лишних (случайных) связей и поэтому мы получаем более правдоподобную картину того, как слова, относящиеся к учебной деятельности, связаны в языке. В первом графе, скорее, результат получался слишком общим: всё связано со всем (сюрприз, язык так работает!), а здесь мы смогли более предметно посмотреть на действительно значимые связи.

### Шаг 6 (1 балл)
Подведите небольшой теоретический итог. Какие выводы о колексификациях в зоне глаголов поиска позволяют сделать эти два графа?

Мы построили и проанализировали графы, отражающие колексификацию глаголов, связанных с процессом обучения, в разных языках. Получается, что самый многозначный глагол, в некотором роде отражающий всё, что связано с обучением, исследованием, проверкой и оценкой знаний - *Analyze*, который, видимо, является почти полным синонимом *study*.

Граф показывает, что в зоне глаголов, связанных с учебным процессом, можно выделить несколько групп близких синонимов, которые можно представить концептами, имеющими в каждой группе наибольший вес: *check* (оценка знаний), *test* (проверка знаний, видимо, включая самопроверку), *examine* (исследование, изучение материала). Многозначность некоторых из слов в каждой группе служит "мостиком", который их объединяет в общую сеть - даже во втором графе все слова, кроме *poll*, связаны.

Таким образом, можно сделать вывод вывод о том, что разные ситуации, связанные с процессом обучения, часто пересекаются, соответствующие слова употребляются рядом и вместе, одно и то же слово может употребляться для описания разных этапов ситуации обучения - что, кажется, соответствует некоторой экстралингвистической интуиции.

### Шаг 7 (бонусный, 1 балл)
Сравните ваши графы с подграфом LOOK FOR из базы CLICS. Что общего, в чем отличия? С чем эти отличия могут быть связаны?

In [20]:
!wget https://raw.githubusercontent.com/clics/clics3/master/clics3-network.gml.zip
!unzip clics3-network.gml.zip

--2024-01-31 20:55:48--  https://raw.githubusercontent.com/clics/clics3/master/clics3-network.gml.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12181049 (12M) [application/zip]
Saving to: ‘clics3-network.gml.zip’

clics3-network.gml. 100%[===================>]  11.62M  --.-KB/s    in 0.1s    

2024-01-31 20:55:48 (116 MB/s) - ‘clics3-network.gml.zip’ saved [12181049/12181049]

Archive:  clics3-network.gml.zip
   creating: graphs/
  inflating: graphs/network-3-families.gml  


In [21]:
with open("graphs/network-3-families.gml", 'r', encoding="utf-8") as file:
    clics_graph_data = file.read()

clics_graph = nx.parse_gml(clics_graph_data)

len(clics_graph)

2919

In [22]:
study_node_id = None
for node in clics_graph.nodes(data=True):
    if node[1].get("Gloss") == "STUDY":
        study_node_id = node[0]
        break

if study_node_id is not None:
    study_subgraph = nx.ego_graph(clics_graph, study_node_id, radius=1)
else:
    study_subgraph = None

print(len(study_subgraph))
[data.get('Gloss') for _, data in study_subgraph.nodes(data=True)]

7


['KNOW (SOMETHING)',
 'UNDERSTAND',
 'READ',
 'IMITATE',
 'TEACH',
 'STUDY',
 'LEARN']

Сюда попали только концепты, относящиеся к процессу получения знаний. Связано это с тем, что сейчас мы рассматриваем только лексемы, связанные с *study* напрямую, тогда как в своём собственном графе мы смотрели на все связи независимо от расстояния. Мы бы могли для CLICS увеличить длину связи, отрегулировав параметр `radius`, однако уже при значении 2 получается много совсем нерелевантных связей и появляются слова вроде *KILL* (что-то в этом есть!). По сути единственное сходство с нашим графом -- концепт *TEACH. Ну и ещё одно важное отличие состоит в том, что тут мы смотрим на значение слова *study*, которое скорее ближе к *learn*, чем к *analyze* - а *analyze* в CLICS нет.

In [23]:
num_connected_components = nx.number_connected_components(study_subgraph)
density = nx.density(study_subgraph)

num_connected_components, density

(1, 0.6190476190476191)

А вот плотность графа достаточно близка к той, которая у нас была в самодельном первом графе. Вероятно, основания для постановки ребер в CLICS достаточно мягкие.

In [24]:
weighted_degrees = dict(study_subgraph.degree(weight='weight'))

{study_subgraph.nodes[key].get("Gloss"): value for key, value in weighted_degrees.items()}

{'KNOW (SOMETHING)': 3,
 'UNDERSTAND': 3,
 'READ': 3,
 'IMITATE': 2,
 'TEACH': 3,
 'STUDY': 6,
 'LEARN': 6}

Ещё одно отличие данного графа от нашего - он не является взвешенным, и веса рёбер просто отражают количество связей с другими концептами. И тут получается, что и *study*, и *learn* связаны со всем остальным - в данном куске они синонимы. У нас это не так, потому что мы скорее работали с другим значением *study*, более близким к *analyze*.

Сообщества в таком графе выделять бесполезно.